1.	Выберите набор данных (датасет) для решения задачи классификации или регресии.
2.	В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3.	С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4.	Обучите следующие ансамблевые модели:

−	одну из моделей группы стекинга.

−	модель многослойного персептрона. По желанию, вместо библиотеки scikit-learn возможно использование библиотек TensorFlow, PyTorch или других аналогичных библиотек.

−	двумя методами на выбор из семейства МГУА (один из линейных методов COMBI / MULTI + один из нелинейных методов MIA / RIA) с использованием библиотеки gmdh.

−	В настоящее время библиотека МГУА не позволяет решать задачу классификации !!!

5.	Оцените качество моделей с помощью одной из подходящих для задачи метрик. Сравните качество полученных моделей.


In [1]:
!python --version

Python 3.11.12


In [2]:
!pip install numpy==1.24.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 89.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is 

In [1]:
!pip install gmdh

In [20]:
!pip install --upgrade pydrive

In [21]:
import warnings

warnings.filterwarnings("ignore", category=ImportWarning)

In [44]:
import gmdh
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [45]:
data = pd.read_csv('/content/sample_data/housing.csv')
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


# Разведочный анализ данных
## Пропуски

In [46]:
data.dtypes

,0
longitude,float64
latitude,float64
housing_median_age,float64
total_rooms,float64
total_bedrooms,float64
population,float64
households,float64
median_income,float64
median_house_value,float64
ocean_proximity,object


In [47]:
data.isnull().sum()

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,207
population,0
households,0
median_income,0
median_house_value,0
ocean_proximity,0


In [48]:
207 / 20640

0.01002906976744186

In [49]:
data = data.dropna(axis=0, how='any')
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [50]:
data['ocean_proximity'].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

## Кодирование ocean_proximity

In [51]:
ohe = OneHotEncoder()
encoded_data = ohe.fit_transform(data[['ocean_proximity']])

In [52]:
# df_sparse = pd.DataFrame.sparse.from_spmatrix(encoded_data)

In [53]:
encoded_data = pd.get_dummies(data, columns=['ocean_proximity'], prefix='ocean').astype(int)
encoded_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_<1H OCEAN,ocean_INLAND,ocean_ISLAND,ocean_NEAR BAY,ocean_NEAR OCEAN
0,-122,37,41,880,129,322,126,8,452600,0,0,0,1,0
1,-122,37,21,7099,1106,2401,1138,8,358500,0,0,0,1,0
2,-122,37,52,1467,190,496,177,7,352100,0,0,0,1,0
3,-122,37,52,1274,235,558,219,5,341300,0,0,0,1,0
4,-122,37,52,1627,280,565,259,3,342200,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121,39,25,1665,374,845,330,1,78100,0,1,0,0,0
20636,-121,39,18,697,150,356,114,2,77100,0,1,0,0,0
20637,-121,39,17,2254,485,1007,433,1,92300,0,1,0,0,0
20638,-121,39,18,1860,409,741,349,1,84700,0,1,0,0,0


## Скейлинг данных

In [54]:
encoded_data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_<1H OCEAN,ocean_INLAND,ocean_ISLAND,ocean_NEAR BAY,ocean_NEAR OCEAN
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.143493,35.108452,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.393383,206864.413155,0.442128,0.317917,0.000245,0.111095,0.128615
std,2.006368,2.142049,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.931455,115435.667099,0.496652,0.465678,0.015641,0.314257,0.334782
min,-124.000000,32.000000,1.000000,2.000000,1.000000,3.000000,1.000000,0.000000,14999.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-121.000000,33.000000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.000000,119500.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-118.000000,34.000000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.000000,179700.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,-118.000000,37.000000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.000000,264700.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,-114.000000,41.000000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000000,500001.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [55]:
X = encoded_data.drop(['median_house_value'], axis=1)

In [56]:
y = encoded_data['median_house_value']

In [57]:
scaler = StandardScaler()
scaler.fit(X)
encoded_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_<1H OCEAN,ocean_INLAND,ocean_ISLAND,ocean_NEAR BAY,ocean_NEAR OCEAN
0,-122,37,41,880,129,322,126,8,452600,0,0,0,1,0
1,-122,37,21,7099,1106,2401,1138,8,358500,0,0,0,1,0
2,-122,37,52,1467,190,496,177,7,352100,0,0,0,1,0
3,-122,37,52,1274,235,558,219,5,341300,0,0,0,1,0
4,-122,37,52,1627,280,565,259,3,342200,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121,39,25,1665,374,845,330,1,78100,0,1,0,0,0
20636,-121,39,18,697,150,356,114,2,77100,0,1,0,0,0
20637,-121,39,17,2254,485,1007,433,1,92300,0,1,0,0,0
20638,-121,39,18,1860,409,741,349,1,84700,0,1,0,0,0


# train_test_split

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Ансамбли
## Стекинг

In [59]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR


# Базовые модели
base_models = [
    ('ridge', Ridge()),
    ('tree', DecisionTreeRegressor()),
    ('svr', SVR())
]

# Мета-модель (финальный предсказатель)
meta_model = Ridge()

# Стекинг
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)
stacking_model.fit(X_train, y_train)

# Оценка
score_st = stacking_model.score(X_test, y_test)




print(f"Score (Stacking): {score_st:.3f}")


Score (Stacking): 0.683


## MLP

In [60]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train, y_train)

print(f"Score (MLP): {mlp.score(X_test, y_test):.3f}")

Score (MLP): 0.708


## МГУА

### Линейный метод Combi

In [61]:
from gmdh import Combi, split_data
import numpy as np

X = X.values
y = y.values.ravel()


X_train_combi, X_test_combi, y_train_combi, y_test_combi = train_test_split(X, y, test_size=0.33, random_state=42)

combi = Combi()
combi.fit(X_train_combi, y_train_combi)
y_pred_com = combi.predict(X_test)

### Нелинейный метод MIA

In [62]:
from gmdh import Mia
import numpy as np

# y_train_np = np.array(y_train).ravel()  # .ravel() для одномерного массива
# y_test_np = np.array(y_test).ravel()
# X_train_np = np.array(X_train).ravel()  # .ravel() для одномерного массива
# X_test_np = np.array(X_test).ravel()

mia = Mia()
mia.fit(X_train_combi, y_train_combi)
y_pred_mia = mia.predict(X_test_combi)

### Оценка

In [63]:
from sklearn.metrics import r2_score

print(f"Score (COMBI): {r2_score(y_test, y_pred_com):.3f}")
print(f"Score (MIA): {r2_score(y_test, y_pred_mia):.3f}")

Score (COMBI): 0.632
Score (MIA): 0.602


In [64]:
print(f"Score (MLP): {mlp.score(X_test, y_test):.3f}")
print(f"Score (Stacking): {score_st:.3f}")

Score (MLP): 0.708
Score (Stacking): 0.683
